In [1]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_parametrizacion_variables
import funcion_SQL_a_DataFrame
import funcion_exportacion_pdf_seriemapas
import funcion_copia_googledrive
import funcion_hectareaje_general_ui

In [2]:

resultado = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_tRendimientosActualizacion = resultado[2]
renombrar_actividades = resultado[4]

In [3]:
df_consulta_tRendimientosActualizacion = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_tRendimientosActualizacion)
print(f"Finaliza Ejecución SQL Radicados Actualización")

Inicio de la conexión
Conexión exitosa ...
Finaliza Ejecución SQL Radicados Actualización


In [4]:
RUTA_CAPA_TERRENOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"

estandarizacion_columnas_tRadActualizacion = ['municipio','componente_responsable','radicado','npn','ficha','actividad_actual_tramite','usuario_actual_responsable_tramite','con_plano_cartografico','tipo_terreno']

fecha_actual = dt.now()
fecha_directorio_w_Rendimiento = str(fecha_actual.strftime("%Y%m%d"))

DIRECTORIO_SALIDA_XLSX = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_2_Rendimientos_Reconocimiento"
NOMBRE_XLSX = fecha_directorio_w_Rendimiento + "_RendimientoEquipoReconocimiento_BCGS.xlsx"
RUTA_SALIDA_XLSX = os.path.join(DIRECTORIO_SALIDA_XLSX, NOMBRE_XLSX)
NOMBRE_HOJA = 'Seguimiento_Avance_NPN_Hito'
NOMBRE_HOJA_AVANCE = 'Porcentaje_Contractual_Avance'

RUTA_HITO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_8_Seguimiento_Avance_Terreno_Hito\Base_Datos\2_2_8_SegAvanceTerrenoHito.gdb"

nLayout = 'Seguimiento Avance Predio X Hito'
rutaPDF = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_8_Seguimiento_Avance_Terreno_Hito\Salida_Grafica"

DIRECTORIO_GLOBAL_LOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_8_Seguimiento_Avance_Terreno_Hito"
DIRECTORIO_GLOBAL_DRIVE = r"G:\Mi unidad\Equipo_Consolidacion\Hitos\E2_Informes_Id_FisicoJuridica\2_2_8_Seguimiento_Avance_Terreno_Hito"

ruta_fcUI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb"


In [5]:
hito = funcion_hectareaje_general_ui.funcion_hectareaje_general_hito(ruta_fcUI)
df_hitos_ut = hito[0]

Se crea dataset de capa UI_Unificadas
Se exporta C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\Estadisticos\indicadores_base_ui_lote_4.xlsx
FINALIZA PROCESO


In [6]:

# # ? Parametrización Actualización Atendiendo
for columna in df_consulta_tRendimientosActualizacion.columns:
    if columna == 'actividad_actual_tramite':
        for llave, valor in renombrar_actividades.items():
            df_consulta_tRendimientosActualizacion.loc[df_consulta_tRendimientosActualizacion[columna]==llave, columna] = valor

df_consulta_tRadicados_Actualizacion = df_consulta_tRendimientosActualizacion[estandarizacion_columnas_tRadActualizacion]

In [7]:

df_terrenos = pd.DataFrame.spatial.from_featureclass(RUTA_CAPA_TERRENOS)

In [8]:
df_seguimiento_avance_X_hito = pd.merge(left=df_terrenos,
    right=df_consulta_tRendimientosActualizacion,
    left_on='codigo',
    right_on='npn',
    how='left')

df_seguimiento_avance_X_hito

df_seguimiento_avance_X_hito = df_seguimiento_avance_X_hito.copy()

df_seguimiento_avance_X_hito['En_Sistema_BCGS'] = None

df_seguimiento_avance_X_hito.loc[df_seguimiento_avance_X_hito['radicado'].notnull(), 'En_Sistema_BCGS'] = 'SI'
df_seguimiento_avance_X_hito.loc[df_seguimiento_avance_X_hito['radicado'].isnull(), 'En_Sistema_BCGS'] = 'NO'

estandarizacion_columnas_seguimiento_avance_sin_GEO = ['codigo','id_ui','meta_hito','area_ha_cmt12','En_Sistema_BCGS','radicado','actividad_actual_tramite','tipo_terreno']
estandarizacion_columnas_seguimiento_avance_con_GEO = ['codigo','id_ui','meta_hito','area_ha_cmt12','En_Sistema_BCGS','radicado','actividad_actual_tramite','tipo_terreno','SHAPE']

df_seguimiento_avance_X_hito_sin_Geo = df_seguimiento_avance_X_hito[estandarizacion_columnas_seguimiento_avance_sin_GEO]
df_seguimiento_avance_X_hito_con_Geo = df_seguimiento_avance_X_hito[estandarizacion_columnas_seguimiento_avance_con_GEO]

In [9]:

df_hitos_ut_contrato = df_hitos_ut[['Hito','Area_Ha_Contractual']]

df_seguimiento_avance_X_hito_enBCGS = df_seguimiento_avance_X_hito_sin_Geo[df_seguimiento_avance_X_hito_sin_Geo['En_Sistema_BCGS'] =='SI']
df_seguimiento_avance_X_hito_enBCGS_Statics = pd.DataFrame(df_seguimiento_avance_X_hito_enBCGS.groupby(['meta_hito'])['area_ha_cmt12'].sum()).reset_index()
df_seguimiento_avance_X_hito_enBCGS_Statics['area_ha_cmt12'] = df_seguimiento_avance_X_hito_enBCGS_Statics['area_ha_cmt12'].round(3)

df_seguimiento_avance_X_hito_enBCGS_Statics_H2 = df_seguimiento_avance_X_hito_enBCGS_Statics[df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 2']
HITO_2 = df_seguimiento_avance_X_hito_enBCGS_Statics_H2['area_ha_cmt12'].sum().round(3)

df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3 = df_seguimiento_avance_X_hito_enBCGS_Statics[(df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 2') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 3')]
HITO2_HITO3 = df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3['area_ha_cmt12'].sum().round(3)

df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3_H4 = df_seguimiento_avance_X_hito_enBCGS_Statics[(df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 2') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 3') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 4')]
HITO2_HITO3_HITO4 = df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3_H4['area_ha_cmt12'].sum().round(3)

df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3_H4_H5 = df_seguimiento_avance_X_hito_enBCGS_Statics[(df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 2') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 3') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 4') |
    (df_seguimiento_avance_X_hito_enBCGS_Statics['meta_hito']=='Hito 5') ]
HITO2_HITO3_HITO4_HITO5 = df_seguimiento_avance_X_hito_enBCGS_Statics_H2_H3_H4_H5['area_ha_cmt12'].sum().round(3)


dict_avance_porcentual = {'Hito 2':HITO_2,
    'Hito 3':HITO2_HITO3,
    'Hito 4':HITO2_HITO3_HITO4,
    'Hito 5':HITO2_HITO3_HITO4_HITO5
    }

renombrar_columnas = {'index':'Hito',
    0:'Area_Avance_X_Hito_CargaEnBCGS'}

df_avance_porcentual_BCGS = pd.DataFrame.from_dict(dict_avance_porcentual, orient='index').reset_index().rename(columns=renombrar_columnas)

df_avance_porcentual_BCGS

,Hito,Area_Avance_X_Hito_CargaEnBCGS
0,Hito 2,13506.042
1,Hito 3,37185.888
2,Hito 4,45995.819
3,Hito 5,45995.819


In [10]:
df_avance_por_hContractual = pd.merge(left=df_hitos_ut_contrato,
    right=df_avance_porcentual_BCGS,
    on='Hito',
    how='left')

df_avance_por_hContractual['%_Avance'] = ((df_avance_por_hContractual['Area_Avance_X_Hito_CargaEnBCGS']/df_avance_por_hContractual['Area_Ha_Contractual'])*100).round(1)

In [11]:
with pd.ExcelWriter(RUTA_SALIDA_XLSX, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_seguimiento_avance_X_hito_sin_Geo.to_excel(writer, sheet_name=NOMBRE_HOJA, index=False)
    df_avance_por_hContractual.to_excel(writer, sheet_name=NOMBRE_HOJA_AVANCE, index=False)

In [12]:
ruta_salida_seguimiento_geo = os.path.join(RUTA_HITO, NOMBRE_HOJA)
df_seguimiento_avance_X_hito_con_Geo.spatial.to_featureclass(location=ruta_salida_seguimiento_geo)
print(f"Se genera la capa {ruta_salida_seguimiento_geo}")

C:\docsProyectos\ENTORNOS\UT\Lib\site-packages\arcgis\features\geo\_accessor.py:1531: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = array


Se genera la capa C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_8_Seguimiento_Avance_Terreno_Hito\Base_Datos\2_2_8_SegAvanceTerrenoHito.gdb\Seguimiento_Avance_NPN_Hito


In [13]:
funcion_exportacion_pdf_seriemapas.exportacion_salida_grafica(nLayout, rutaPDF)

Nombre Layout: Seguimiento Avance Predio X Hito
Se exporta a PDF Seguimiento Avance Predio X Hito


In [14]:
funcion_copia_googledrive.migracion_googledrive(DIRECTORIO_GLOBAL_LOCAL, DIRECTORIO_GLOBAL_DRIVE)

Se copia BD 2_2_8_SegAvanceTerrenoHito.gdb
Se copia archivo PDF 20240717_María La Baja.pdf
Se copia archivo PDF 20240717_Repelón.pdf
Se copia archivo PDF 20240718_María La Baja.pdf
Se copia archivo PDF 20240718_Repelón.pdf
Se copia archivo PDF 20240719_María La Baja.pdf
Se copia archivo PDF 20240719_Repelón.pdf
Se copia archivo PDF 20240722_María La Baja.pdf
Se copia archivo PDF 20240722_Repelón.pdf
Se copia archivo PDF 20240723_María La Baja.pdf
Se copia archivo PDF 20240723_Repelón.pdf
Se copia archivo PDF 20240724_María La Baja.pdf
Se copia archivo PDF 20240724_Repelón.pdf
Se copia archivo PDF 20240725_María La Baja.pdf
Se copia archivo PDF 20240725_Repelón.pdf
Se copia archivo PDF 20240726_María La Baja.pdf
Se copia archivo PDF 20240726_Repelón.pdf
Se copia archivo PDF 20240727_María La Baja.pdf
Se copia archivo PDF 20240727_Repelón.pdf
Se copia archivo PDF 20240729_María La Baja.pdf
Se copia archivo PDF 20240729_Repelón.pdf
Se copia archivo PDF 20240730_María La Baja.pdf
Se copia 